In [2]:
import requests
import pandas as pd

In [8]:
resp = requests.get(
    'https://api.kraken.com/0/public/OHLC?pair=XBTUSD,interval=3600')
resp


<Response [200]>

In [9]:
print(resp.json())

{'error': ['EQuery:Unknown asset pair']}
